In [5]:
import rasterio
from rasterio.mask import mask
from rasterio import Affine
import geopandas as gpd
from shapely.geometry import box
from rasterio.features import geometry_mask
import numpy as np
from shapely.geometry import mapping
from shapely.geometry import Polygon
import os
from osgeo import gdal,ogr,osr

In [7]:
raster1 = rasterio.open(r'F:\wenqu\trait_map\trait_proj\site2c_d13c_trait_project_to_aviris.tif')
raster2 = rasterio.open(r'F:/wenqu/trait_map/trait_clip/site2c_d13c/26_14.tif')
shapefile = gpd.read_file('F:/wenqu/trait_map/output.shp')
geom = shapefile.geometry

out_img, out_transform = mask(raster1, geom, crop=True)



# Update the metadata of the output image:
out_meta = raster1.meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": out_img.shape[1],
    "width": out_img.shape[2],
    "transform": out_transform
})

output = 'F:/wenqu/trait_map/trait_clip/site2c_d13c/' + '26_14_1.tif'

# Write the clipped raster to a new tif file:
with rasterio.open(output, "w", **out_meta) as dest:
    for band in range(1, raster1.count + 1):
        dest.write(out_img[band-1], indexes=band)

In [ ]:
raster1 = rasterio.open(r'F:\wenqu\trait_map\site2a_la_trait_map_project3.tif')

## use shp file to clip trait image to the exactly same extent as the aviris pixel

# this is to use rasterio to clip


# if shp and ti doesn't overlap, move to next one

In [9]:
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask

# Paths
shapefile_folder = r"D:\wenqu\chapter1_2\updated_aviris\aviris_ng_shp\site6"
raster_path = r"D:\wenqu\chapter1_2\updated_uas_modeling\uas_trait_map\projection\site6_pc_reprojected.tif"
output_folder = r"D:\wenqu\chapter1_2\updated_uas_modeling\trait_clip\pc"

os.makedirs(output_folder, exist_ok=True)

# Load raster
with rasterio.open(raster_path) as src:
    raster_bounds = src.bounds
    raster_crs = src.crs

# Loop through all shapefiles
for file in os.listdir(shapefile_folder):
    if file.endswith(".shp"):
        shp_path = os.path.join(shapefile_folder, file)
        gdf = gpd.read_file(shp_path)

        # Reproject shapefile to raster CRS
        if gdf.crs != raster_crs:
            gdf = gdf.to_crs(raster_crs)

        # Check intersection
        shp_bounds = gdf.total_bounds
        if (shp_bounds[2] < raster_bounds.left or shp_bounds[0] > raster_bounds.right or
            shp_bounds[3] < raster_bounds.bottom or shp_bounds[1] > raster_bounds.top):
            print(f"❌ Skipped (no overlap): {file}")
            continue

        with rasterio.open(raster_path) as src:
            try:
                out_image, out_transform = mask(src, gdf.geometry, crop=True)
            except ValueError:
                print(f"⚠️ Geometry error, skipping: {file}")
                continue

            out_meta = src.meta.copy()
            out_meta.update({
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
                "crs": raster_crs
            })

            output_name = os.path.splitext(file)[0] + "_clipped.tif"
            output_path = os.path.join(output_folder, output_name)

            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)

            print(f"✅ Clipped and saved: {output_name}")


❌ Skipped (no overlap): site6_000_000.shp
❌ Skipped (no overlap): site6_000_001.shp
❌ Skipped (no overlap): site6_000_002.shp
❌ Skipped (no overlap): site6_000_003.shp
❌ Skipped (no overlap): site6_000_004.shp
❌ Skipped (no overlap): site6_000_005.shp
✅ Clipped and saved: site6_000_006_clipped.tif
✅ Clipped and saved: site6_000_007_clipped.tif
✅ Clipped and saved: site6_000_008_clipped.tif
✅ Clipped and saved: site6_000_009_clipped.tif
✅ Clipped and saved: site6_000_010_clipped.tif
❌ Skipped (no overlap): site6_000_011.shp
❌ Skipped (no overlap): site6_000_012.shp
❌ Skipped (no overlap): site6_000_013.shp
❌ Skipped (no overlap): site6_000_014.shp
❌ Skipped (no overlap): site6_000_015.shp
❌ Skipped (no overlap): site6_000_016.shp
❌ Skipped (no overlap): site6_000_017.shp
❌ Skipped (no overlap): site6_000_018.shp
❌ Skipped (no overlap): site6_000_019.shp
❌ Skipped (no overlap): site6_000_020.shp
❌ Skipped (no overlap): site6_000_021.shp
❌ Skipped (no overlap): site6_000_022.shp
❌ Skipped

✅ Clipped and saved: site6_003_008_clipped.tif
✅ Clipped and saved: site6_003_009_clipped.tif
✅ Clipped and saved: site6_003_010_clipped.tif
✅ Clipped and saved: site6_003_011_clipped.tif
✅ Clipped and saved: site6_003_012_clipped.tif
✅ Clipped and saved: site6_003_013_clipped.tif
✅ Clipped and saved: site6_003_014_clipped.tif
✅ Clipped and saved: site6_003_015_clipped.tif
✅ Clipped and saved: site6_003_016_clipped.tif
✅ Clipped and saved: site6_003_017_clipped.tif
✅ Clipped and saved: site6_003_018_clipped.tif
✅ Clipped and saved: site6_003_019_clipped.tif
✅ Clipped and saved: site6_003_020_clipped.tif
✅ Clipped and saved: site6_003_021_clipped.tif
✅ Clipped and saved: site6_003_022_clipped.tif
✅ Clipped and saved: site6_003_023_clipped.tif
✅ Clipped and saved: site6_003_024_clipped.tif
✅ Clipped and saved: site6_003_025_clipped.tif
✅ Clipped and saved: site6_003_026_clipped.tif
✅ Clipped and saved: site6_003_027_clipped.tif
✅ Clipped and saved: site6_003_028_clipped.tif
✅ Clipped and

✅ Clipped and saved: site6_006_007_clipped.tif
✅ Clipped and saved: site6_006_008_clipped.tif
✅ Clipped and saved: site6_006_009_clipped.tif
✅ Clipped and saved: site6_006_010_clipped.tif
✅ Clipped and saved: site6_006_011_clipped.tif
✅ Clipped and saved: site6_006_012_clipped.tif
✅ Clipped and saved: site6_006_013_clipped.tif
✅ Clipped and saved: site6_006_014_clipped.tif
✅ Clipped and saved: site6_006_015_clipped.tif
✅ Clipped and saved: site6_006_016_clipped.tif
✅ Clipped and saved: site6_006_017_clipped.tif
✅ Clipped and saved: site6_006_018_clipped.tif
✅ Clipped and saved: site6_006_019_clipped.tif
✅ Clipped and saved: site6_006_020_clipped.tif
✅ Clipped and saved: site6_006_021_clipped.tif
✅ Clipped and saved: site6_006_022_clipped.tif
✅ Clipped and saved: site6_006_023_clipped.tif
✅ Clipped and saved: site6_006_024_clipped.tif
✅ Clipped and saved: site6_006_025_clipped.tif
✅ Clipped and saved: site6_006_026_clipped.tif
✅ Clipped and saved: site6_006_027_clipped.tif
✅ Clipped and

✅ Clipped and saved: site6_008_059_clipped.tif
✅ Clipped and saved: site6_008_060_clipped.tif
✅ Clipped and saved: site6_008_061_clipped.tif
❌ Skipped (no overlap): site6_009_000.shp
❌ Skipped (no overlap): site6_009_001.shp
❌ Skipped (no overlap): site6_009_002.shp
❌ Skipped (no overlap): site6_009_003.shp
❌ Skipped (no overlap): site6_009_004.shp
✅ Clipped and saved: site6_009_005_clipped.tif
✅ Clipped and saved: site6_009_006_clipped.tif
✅ Clipped and saved: site6_009_007_clipped.tif
✅ Clipped and saved: site6_009_008_clipped.tif
✅ Clipped and saved: site6_009_009_clipped.tif
✅ Clipped and saved: site6_009_010_clipped.tif
✅ Clipped and saved: site6_009_011_clipped.tif
✅ Clipped and saved: site6_009_012_clipped.tif
✅ Clipped and saved: site6_009_013_clipped.tif
✅ Clipped and saved: site6_009_014_clipped.tif
✅ Clipped and saved: site6_009_015_clipped.tif
✅ Clipped and saved: site6_009_016_clipped.tif
✅ Clipped and saved: site6_009_017_clipped.tif
✅ Clipped and saved: site6_009_018_cli

✅ Clipped and saved: site6_011_050_clipped.tif
✅ Clipped and saved: site6_011_051_clipped.tif
✅ Clipped and saved: site6_011_052_clipped.tif
✅ Clipped and saved: site6_011_053_clipped.tif
✅ Clipped and saved: site6_011_054_clipped.tif
✅ Clipped and saved: site6_011_055_clipped.tif
✅ Clipped and saved: site6_011_056_clipped.tif
✅ Clipped and saved: site6_011_057_clipped.tif
✅ Clipped and saved: site6_011_058_clipped.tif
✅ Clipped and saved: site6_011_059_clipped.tif
✅ Clipped and saved: site6_011_060_clipped.tif
✅ Clipped and saved: site6_011_061_clipped.tif
❌ Skipped (no overlap): site6_012_000.shp
❌ Skipped (no overlap): site6_012_001.shp
❌ Skipped (no overlap): site6_012_002.shp
❌ Skipped (no overlap): site6_012_003.shp
❌ Skipped (no overlap): site6_012_004.shp
✅ Clipped and saved: site6_012_005_clipped.tif
✅ Clipped and saved: site6_012_006_clipped.tif
✅ Clipped and saved: site6_012_007_clipped.tif
✅ Clipped and saved: site6_012_008_clipped.tif
✅ Clipped and saved: site6_012_009_cli

...

✅ Clipped and saved: site6_048_056_clipped.tif
✅ Clipped and saved: site6_048_057_clipped.tif
❌ Skipped (no overlap): site6_048_058.shp
❌ Skipped (no overlap): site6_048_059.shp
❌ Skipped (no overlap): site6_048_060.shp
❌ Skipped (no overlap): site6_048_061.shp
❌ Skipped (no overlap): site6_049_000.shp
✅ Clipped and saved: site6_049_001_clipped.tif
✅ Clipped and saved: site6_049_002_clipped.tif
✅ Clipped and saved: site6_049_003_clipped.tif
✅ Clipped and saved: site6_049_004_clipped.tif
✅ Clipped and saved: site6_049_005_clipped.tif
✅ Clipped and saved: site6_049_006_clipped.tif
✅ Clipped and saved: site6_049_007_clipped.tif
✅ Clipped and saved: site6_049_008_clipped.tif
✅ Clipped and saved: site6_049_009_clipped.tif
✅ Clipped and saved: site6_049_010_clipped.tif
✅ Clipped and saved: site6_049_011_clipped.tif
✅ Clipped and saved: site6_049_012_clipped.tif
✅ Clipped and saved: site6_049_013_clipped.tif
✅ Clipped and saved: site6_049_014_clipped.tif
✅ Clipped and saved: site6_049_015_cli

✅ Clipped and saved: site6_051_046_clipped.tif
✅ Clipped and saved: site6_051_047_clipped.tif
✅ Clipped and saved: site6_051_048_clipped.tif
✅ Clipped and saved: site6_051_049_clipped.tif
✅ Clipped and saved: site6_051_050_clipped.tif
✅ Clipped and saved: site6_051_051_clipped.tif
✅ Clipped and saved: site6_051_052_clipped.tif
✅ Clipped and saved: site6_051_053_clipped.tif
✅ Clipped and saved: site6_051_054_clipped.tif
✅ Clipped and saved: site6_051_055_clipped.tif
✅ Clipped and saved: site6_051_056_clipped.tif
✅ Clipped and saved: site6_051_057_clipped.tif
❌ Skipped (no overlap): site6_051_058.shp
❌ Skipped (no overlap): site6_051_059.shp
❌ Skipped (no overlap): site6_051_060.shp
❌ Skipped (no overlap): site6_051_061.shp
❌ Skipped (no overlap): site6_052_000.shp
✅ Clipped and saved: site6_052_001_clipped.tif
✅ Clipped and saved: site6_052_002_clipped.tif
✅ Clipped and saved: site6_052_003_clipped.tif
✅ Clipped and saved: site6_052_004_clipped.tif
✅ Clipped and saved: site6_052_005_cli

✅ Clipped and saved: site6_054_039_clipped.tif
✅ Clipped and saved: site6_054_040_clipped.tif
✅ Clipped and saved: site6_054_041_clipped.tif
✅ Clipped and saved: site6_054_042_clipped.tif
✅ Clipped and saved: site6_054_043_clipped.tif
✅ Clipped and saved: site6_054_044_clipped.tif
✅ Clipped and saved: site6_054_045_clipped.tif
✅ Clipped and saved: site6_054_046_clipped.tif
✅ Clipped and saved: site6_054_047_clipped.tif
✅ Clipped and saved: site6_054_048_clipped.tif
✅ Clipped and saved: site6_054_049_clipped.tif
✅ Clipped and saved: site6_054_050_clipped.tif
✅ Clipped and saved: site6_054_051_clipped.tif
✅ Clipped and saved: site6_054_052_clipped.tif
✅ Clipped and saved: site6_054_053_clipped.tif
✅ Clipped and saved: site6_054_054_clipped.tif
✅ Clipped and saved: site6_054_055_clipped.tif
✅ Clipped and saved: site6_054_056_clipped.tif
❌ Skipped (no overlap): site6_054_057.shp
❌ Skipped (no overlap): site6_054_058.shp
❌ Skipped (no overlap): site6_054_059.shp
❌ Skipped (no overlap): site

✅ Clipped and saved: site6_057_030_clipped.tif
✅ Clipped and saved: site6_057_031_clipped.tif
✅ Clipped and saved: site6_057_032_clipped.tif
✅ Clipped and saved: site6_057_033_clipped.tif
✅ Clipped and saved: site6_057_034_clipped.tif
✅ Clipped and saved: site6_057_035_clipped.tif
✅ Clipped and saved: site6_057_036_clipped.tif
✅ Clipped and saved: site6_057_037_clipped.tif
✅ Clipped and saved: site6_057_038_clipped.tif
✅ Clipped and saved: site6_057_039_clipped.tif
✅ Clipped and saved: site6_057_040_clipped.tif
✅ Clipped and saved: site6_057_041_clipped.tif
✅ Clipped and saved: site6_057_042_clipped.tif
✅ Clipped and saved: site6_057_043_clipped.tif
✅ Clipped and saved: site6_057_044_clipped.tif
✅ Clipped and saved: site6_057_045_clipped.tif
✅ Clipped and saved: site6_057_046_clipped.tif
✅ Clipped and saved: site6_057_047_clipped.tif
✅ Clipped and saved: site6_057_048_clipped.tif
✅ Clipped and saved: site6_057_049_clipped.tif
✅ Clipped and saved: site6_057_050_clipped.tif
✅ Clipped and

✅ Clipped and saved: site6_060_028_clipped.tif
✅ Clipped and saved: site6_060_029_clipped.tif
✅ Clipped and saved: site6_060_030_clipped.tif
✅ Clipped and saved: site6_060_031_clipped.tif
✅ Clipped and saved: site6_060_032_clipped.tif
✅ Clipped and saved: site6_060_033_clipped.tif
✅ Clipped and saved: site6_060_034_clipped.tif
✅ Clipped and saved: site6_060_035_clipped.tif
✅ Clipped and saved: site6_060_036_clipped.tif
✅ Clipped and saved: site6_060_037_clipped.tif
✅ Clipped and saved: site6_060_038_clipped.tif
✅ Clipped and saved: site6_060_039_clipped.tif
✅ Clipped and saved: site6_060_040_clipped.tif
✅ Clipped and saved: site6_060_041_clipped.tif
✅ Clipped and saved: site6_060_042_clipped.tif
✅ Clipped and saved: site6_060_043_clipped.tif
✅ Clipped and saved: site6_060_044_clipped.tif
✅ Clipped and saved: site6_060_045_clipped.tif
✅ Clipped and saved: site6_060_046_clipped.tif
✅ Clipped and saved: site6_060_047_clipped.tif
✅ Clipped and saved: site6_060_048_clipped.tif
✅ Clipped and

In [12]:
folder_path = r'E:\wenqu\trait_map\update1\trait_map_with_scale\validation\validation1\validation_shapefile\site6'
extension = ".shp"
raster1 = rasterio.open(r"E:\wenqu\trait_map\update1\trait_map_with_scale\training\training1\aviris_trait_map\site6_d13c_std.tif")

for filename in os.listdir(folder_path):
    if filename.endswith(extension) and os.path.isfile(os.path.join(folder_path, filename)):
        trait_name = os.path.splitext(filename)[0] + '.tif'
        print(trait_name)

        shapefile = gpd.read_file(os.path.join(folder_path, filename))
        geom = shapefile.geometry
        out_img, out_transform = mask(raster1, geom, crop=True)

        # Convert the output image to float type to support NaN
        out_img = out_img.astype('float32')

        # Replace 0.0 values with NaN
        out_img[out_img == 0.0] = np.nan

        # Update the metadata of the output image
        out_meta = raster1.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform,
            # Setting datatype to float32 to support NaN
            "dtype": 'float32',
            "nodata": None  # NaN is default nodata for floating point rasters
        })
        
        output = os.path.join(r'E:\wenqu\trait_map\update1\trait_map_with_scale\validation\validation1\from_aviris\d13c\std\site6', 'd13c_aviris_' + trait_name)
        
        # Write the clipped raster to a new tif file
        with rasterio.open(output, "w", **out_meta) as dest:
            dest.write(out_img)


site6_35_027.tif
site6_40_022.tif
site6_47_017.tif
site6_53_012.tif


# this is to use gdal to clip


In [3]:
from osgeo import gdal, ogr

folder_path = r'F:\wenqu\Aviris\2022\pixel_shp\site1b_shp'
extension = ".shp"
input_raster_path = r"F:\wenqu\Aviris\2022\trait_map\trait_map_projection\site1b_d15n_trait_proj.tif"
output_folder = r'F:/wenqu/Aviris/2022/trait_map/site1b_d15n/'

# Enable GDAL exceptions
gdal.UseExceptions()

def raster_bbox(raster_path):
    ds = gdal.Open(raster_path)
    ulx, xres, xskew, uly, yskew, yres = ds.GetGeoTransform()
    lrx = ulx + (ds.RasterXSize * xres)
    lry = uly + (ds.RasterYSize * yres)
    return ulx, lry, lrx, uly  # Return as xmin, ymin, xmax, ymax

def shapefile_bbox(shape_path):
    driver = ogr.GetDriverByName('ESRI Shapefile')
    dataSource = driver.Open(shape_path, 0)
    layer = dataSource.GetLayer()
    return layer.GetExtent()  # Returns as xmin, xmax, ymin, ymax

def intersect_bbox(bbox1, bbox2):
    return not (bbox1[2] < bbox2[0] or bbox1[0] > bbox2[2] or bbox1[3] < bbox2[1] or bbox1[1] > bbox2[3])

raster_bounds = raster_bbox(input_raster_path)

for filename in os.listdir(folder_path):
    if filename.endswith(extension):
        shapefile_path = os.path.join(folder_path, filename)
        shapefile_bounds = shapefile_bbox(shapefile_path)

        if not intersect_bbox(raster_bounds, shapefile_bounds):
            print(f"Shapefile {filename} does not overlap with raster. Skipping...")
            continue

        # Define the name of the output TIF
        output_name = os.path.splitext(filename)[0] + '.tif'
        output_path = os.path.join(output_folder, output_name)

        # Clip the raster using the shapefile and save as a new TIF
        try:
            ds = gdal.Warp(output_path, input_raster_path, cutlineDSName=shapefile_path, cropToCutline=True)
            ds = None  # Close the dataset
            print(f"Saved clipped raster to {output_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

print("Processing completed.")


Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_0.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_1.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_10.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_11.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_12.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_13.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_14.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_15.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_16.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_17.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_18.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_0_19.tif
Saved 

Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_100_23.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_100_24.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_100_25.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_100_26.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_100_27.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_100_28.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_100_29.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_100_3.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_100_30.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_100_31.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_100_32.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n

Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_101_22.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_101_23.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_101_24.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_101_25.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_101_26.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_101_27.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_101_28.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_101_29.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_101_3.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_101_30.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_101_31.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n

Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_102_28.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_102_29.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_102_3.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_102_30.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_102_31.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_102_32.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_102_33.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_102_34.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_102_35.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_102_36.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_102_37.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n

Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_103_29.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_103_3.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_103_30.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_103_31.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_103_32.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_103_33.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_103_34.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_103_35.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_103_36.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_103_37.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_103_38.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n

...

Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_5.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_50.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_51.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_52.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_53.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_54.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_55.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_56.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_57.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_58.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_59.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_96_

Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_50.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_51.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_52.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_53.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_54.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_55.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_56.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_57.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_58.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_59.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_6.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_97_

Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_50.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_51.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_52.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_53.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_54.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_55.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_56.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_57.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_58.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_59.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_6.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_98_

Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_56.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_57.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_58.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_59.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_6.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_60.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_61.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_62.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_63.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_64.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_65.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_99_

Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_9_57.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_9_58.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_9_59.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_9_6.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_9_60.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_9_61.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_9_62.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_9_63.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_9_64.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_9_65.tif
Saved clipped raster to F:/wenqu/Aviris/2022/trait_map/site1b_d15n/site1b_9_66.tif
Shapefile site1b_9_67.shp does not overlap with raster. Skipping...
Shapefile site1b_9_6

## this is to use the aviris pixel bounding box to clip the trait image

In [2]:
raster1 = rasterio.open(r'F:\wenqu\trait_map\trait_proj\site2a_d15n_trait_project_to_aviris.tif')
raster2 = rasterio.open(r'F:\wenqu\Aviris\site2a\PLSR\aviris_single_pixel1\site2a_11_14.tif')

In [3]:
# Define the mask based on the non-zero values of raster1
# mask = data1 != 0
# print(type(mask), mask.shape)
# Define the extent of raster2 as a bounding box:
xmin, ymin, xmax, ymax = raster2.bounds
bbox = box(xmin, ymin, xmax, ymax)
print(type(bbox))

<class 'shapely.geometry.polygon.Polygon'>


In [4]:
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0])
geo = geo.set_crs(raster2.crs)
# geo = geo.to_crs(raster1.crs)
print(type(geo))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [5]:
print(geo)

                                            geometry
0  POLYGON ((624228.242 7476897.110, 624228.242 7...


In [6]:
# Clip raster1 using the extent of raster2:
out_img, out_transform = mask(raster1, shapes=geo.geometry, crop=True)

# Update the metadata of the output image:
out_meta = raster1.meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": out_img.shape[1],
    "width": out_img.shape[2],
    "transform": out_transform
})

output = 'F:/wenqu/trait_map/trait_clip/site2a_d13c_test/' + 'site2a_11_14.tif'

# Write the clipped raster to a new tif file:
with rasterio.open(output, "w", **out_meta) as dest:
    for band in range(1, raster1.count + 1):
        dest.write(out_img[band-1], indexes=band)

In [3]:
import os

folder_path = "F:/wenqu/Aviris/site2a/PLSR/aviris_single_pixel"
extension = ".tif"

for filename in os.listdir(folder_path):
    # Check if the current item is a file
    if filename.endswith(extension) and os.path.isfile(os.path.join(folder_path, filename)):
        # Do something with the file, e.g. print its name
        print(filename)
        raster1 = rasterio.open(r'F:\wenqu\trait_map\site2a_la_trait_map_project3.tif')
        
        raster2 = rasterio.open('F:/wenqu/Aviris/site2a/PLSR/aviris_single_pixel/' + filename) 
        xmin, ymin, xmax, ymax = raster2.bounds
        bbox = box(xmin, ymin, xmax, ymax)
        # Reproject the bounding box of raster2 to the coordinate system of raster1:
        geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0])
        geo = geo.set_crs(raster2.crs)
#         geo = geo.to_crs(raster1.crs)
        
        # Clip raster1 using the extent of raster2:
        out_img, out_transform = mask(raster1, shapes=geo.geometry, crop=True)

        # Update the metadata of the output image:
        out_meta = raster1.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform
        })
        
        output = 'F:/wenqu/trait_map/clipped_trait_map/' + filename

        # Write the clipped raster to a new tif file:
        with rasterio.open(output, "w", **out_meta) as dest:
            dest.write(out_img)


site2a_0_11.tif


RasterioIOError: Attempt to create new tiff file 'F:/wenqu/trait_map/clipped_trait_map/site2a_0_11.tif' failed: No such file or directory